In [ ]:
import json
import os
import pandas as pd
from pathlib import Path
import torch
from transformers import AutoModelForCausalLM
from typing import List

## Download or load model

In [ ]:
# Define a central location for storing models
CENTRAL_MODEL_DIR = os.path.expanduser('~/huggingface_models')

# model_name = 'microsoft/phi-2'
# model_name = 'microsoft/phi-1_5'
# model_name = 'microsoft/Phi-3.5-mini-instruct'
# model_name = 'google/gemma-2-9b'
# model_name = 'meta-llama/Meta-Llama-3.1-8B'
# model_name = 'meta-llama/Meta-Llama-3.1-8B-Instruct'
model_name = 'google/gemma-2-2b-it'
# model_name = 'google/gemma-2-9b-it'

# Create the central directory if it doesn't exist
os.makedirs(CENTRAL_MODEL_DIR, exist_ok=True)

# Define the path where the model will be saved locally
local_model_path = os.path.join(CENTRAL_MODEL_DIR, model_name.replace('/', '-'))

In [ ]:
# Automatically detect and use GPU if available, otherwise use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Set up the device map
if torch.cuda.is_available():
    device_map = "auto"  # This will automatically distribute the model across available GPUs
else:
    device_map = {"": device}  # Use the detected device (CPU in this case)

In [ ]:
# Check if the model exists locally
if os.path.exists(local_model_path):
    print(f"Loading model from local path: {local_model_path}")
    original_model = AutoModelForCausalLM.from_pretrained(
        local_model_path,
        device_map=device_map,
        # quantization_config=bnb_config,
        trust_remote_code=True
    )
else:
    print(f"Downloading model from {model_name}")
    original_model = AutoModelForCausalLM.from_pretrained(
        model_name,
        device_map=device_map,
        # quantization_config=bnb_config,
        trust_remote_code=True
    )
    # Save the model locally
    original_model.save_pretrained(local_model_path)
    print(f"Model saved to {local_model_path}")

## Load the dataset

In [ ]:
df = pd.read_csv('../data/ReportsDATASET.csv')

## Define the labeling prompt

In [ ]:
def classify_abnormalities(abnormalities: List[str], report: str, llm_function) -> str:
    # Create a dynamic prompt for the LLM
    prompt = f"""
Given the following radiology report, classify the presence (1) or absence (0) of the specified abnormalities.
Output the result as a JSON string without any additional explanation.

Abnormalities to classify: {', '.join(abnormalities)}

Radiology report:
{report}

Output format:
{{
    "abnormality1": 0 or 1,
    "abnormality2": 0 or 1,
    ...
}}

The output must be a JSON string without any explanation.
"""

    # Call the LLM function with the prompt
    llm_output = llm_function(prompt)

    # Ensure the output is valid JSON
    try:
        result = json.loads(llm_output)
        # Verify that all abnormalities are present in the output
        for abnormality in abnormalities:
            if abnormality not in result:
                raise ValueError(f"Missing abnormality in LLM output: {abnormality}")
        return json.dumps(result)
    except json.JSONDecodeError:
        raise ValueError("LLM output is not valid JSON")
    except Exception as e:
        raise ValueError(f"Error processing LLM output: {str(e)}")

# Example usage 
def mock_llm_function(prompt: str) -> str:
    # This is a mock function that simulates an LLM's response
    return '{"pulmonary edema": 1, "consolidation": 0, "pleural effusion": 1, "pneumothorax": 0, "cardiomegaly": 1}'

# Example usage
abnormalities = ["pulmonary edema", "consolidation", "pleural effusion", "pneumothorax", "cardiomegaly"]
report = "The chest radiograph shows increased opacification in the lower lung fields bilaterally, consistent with pulmonary edema. There is also evidence of pleural effusion and an enlarged cardiac silhouette suggestive of cardiomegaly."

result = classify_abnormalities(abnormalities, report, mock_llm_function)
print(result)